# Защита персональных данных клиентов

Страховой компании «Хоть потоп» необходимо защитить персональные данные своих клиентов. Для защиты данных предлагается разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. При преобразовании качество моделей машинного обучения не должно ухудшиться.

**Цель исследования**: преобразовать данные клиентов, чтобы качество моделей машинного обучения не ухудшилось.

**Ход исследования**

Данные для исследования получим из файла `insurance.csv`.

Исследование пройдет в 4 этапа:
1. Загрузка и изучение данных
2. Предложение алгоритма преобразования данных
3. Проверка предложенного алгоритма

## Загрузка данных

Для начала импортируем необходимые в работе библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Прочитаем файл `insurance.csv` из каталога `datasets` и сохраним его в одноименной переменной.

In [2]:
insurance = pd.read_csv('/datasets/insurance.csv')

Составим первое впечатление о данных:
1. Получим общую информацию методом `info()`.
2. Выведем на экран первые пять строк таблицы методом `head()`. 
3. Оценим разброс значений методом `describe()`.

In [3]:
insurance.info()
display(insurance.head())
display(insurance.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Итак, в таблице 5 столбцов: 4 из них содержат информацию о застрахованном лице (пол, возраст, зарплата, количество членов семьи), в последнем столбце содержатся сведения о количестве страховых выплат клиенту за последние пять лет.

Пропусков в данных нет, но признаки `Возраст` и `Зарплата` лучше привести к целочисленному типу. Для экономии памяти будем использовать тип данных int32 вместо int64.

In [4]:
insurance = insurance.astype('int32')

### Выводы

Исходные данные содержат информацию о клиенте (пол, возраст, зарплата, количество членов семьи) и целевой признак: Страховые выплаты. На следующем этапе исследования предложим алгоритм преобразования исходных признаков, чтобы качество модели обучения, предсказывающей значение целевого признака, не ухудшилось.

## Алгоритм преобразования

Попробуем преобразовать данные путем умножения исходных признаков на обратимую матрицу. Проверим, что при таком преобразовании данных качество модели линейной регрессии не изменится. Воспользуемся формулами линейной регрессии.

Введем обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

При умножении признаков на обратимую матрицу формула обучения будет выглядить следующим образом:
    
$$
w = ((XP)^T XP)^{-1} (XP)^T y
$$

Воспользуемся свойством транспортированной матрицы $(AB)^T = B^T A^T$:

$$
w = (P^T X^T XP)^{-1} (XP)^T y
$$

Используя свойство обратной матрицы $(AB)^{-1} = B^{-1}A^{-1}$ преобразуем равенство:

$$
w = P^{-1}(P^T X^T X)^{-1} (XP)^T y
$$

$$
w = P^{-1}(X^T X)^{-1} (P^T)^{-1} (XP)^T y
$$

Еще раз обратимся к свойству обратной матрицы $(AB)^T = B^T A^T$:

$$
w = P^{-1}(X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

При умножении обратной матрицы на исходную получится единичная матрица $E$:

$$
w = P^{-1}(X^T X)^{-1} E X^T y
$$

Умножая матрицу на единичную, исходная матрица не меняется:

$$
w = P^{-1}(X^T X)^{-1} X^T y
$$

Подставим получившееся значение в формулу предсказания:

$$
a = XPP^{-1}(X^T X)^{-1} X^T y
$$

$$
a = XE(X^T X)^{-1} X^T y
$$

$$
a = X(X^T X)^{-1} X^T y
$$

Один из множителей является формулой обучения $w = (X^T X)^{-1} X^T y$, таким образом, конечное равенство имеет вид $a = Xw$.  

После всех преобразований получили формулу предсказания $a = Xw$, на основе чего делаем вывод, что при умножении признаков на обратимую матрицу качество модели линейной регрессии не изменится.

Стоит помнить о том, что умножать матрицу размера m * n можно только на матрицу n * k, поэтому размер матрицы $P$ должен быть равен количеству столбцов матрицы $X$ . 

Произвольную матрицу необходимого размера получим функцией `random.rand()`, обратимость матрицы проверим с помощью функции `linalg.inv()` библиотеки `numpy`.

Напишем функцию преобразования исходных признаков `transform_features()`.

In [5]:
def transform_features(features):
    n = features.shape[1]
    np.random.seed(100)
    rand_matrix = np.random.rand(n,n)
    rand_matrix_inv = []
    while len(rand_matrix_inv) == 0:
        try:
            rand_matrix_inv = np.linalg.inv(rand_matrix)
        except:
            rand_matrix = np.random.rand(n,n)
    return np.dot(features, rand_matrix)

### Выводы

В качестве алгоритма преобразования данных было предложено умножать исходные признаки таблицы на обратимую матрицу. Проверка показала, что такой способ преобразования не влияет на качество модели обучения. Чтобы преобразование выполнялось, размер обратимой матрицы должен быть равен количеству признаков исходной таблицы.

На следующем шаге проверим алгоритм преобразования на практических данных.

## Проверка алгоритма

Целевым признаком таблицы являются `Страховые выплаты`: запишем его в переменную `target`. Остальные признаки таблицы сохраним в `features`.

In [6]:
target = insurance['Страховые выплаты']
features = insurance.drop('Страховые выплаты', axis=1)

Обучим модель линейной регрессии на исходных данных и проверим качество обучение по метрике $R2$.

In [7]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(f'R2-метрика на исходных данных: {r2_score(target, predictions)}')

R2-метрика на исходных данных: 0.42494550308169177


Преобразуем исходные признаки, используя функцию `transform_features`, затем обучим модель линейной регрессии и найдем $R2$-меру.

In [8]:
transform_features = transform_features(features)

model.fit(transform_features, target)
predictions = model.predict(transform_features)
print(f'R2-метрика на преобразованных данных: {r2_score(target, predictions)}')

R2-метрика на преобразованных данных: 0.4249455030817211


### Выводы

Значение R2-метрики модели линейной регресси на исходных и преобразованных данных совпадает. Таким образом, алгоритм преобразования данных путем умножения исходных признаков на обратимую матрицу не влияет на качество модели обучения, и, как следствие, может быть использован для решения задачи защиты персональных данных клиентов.

## Общий вывод

Для решения задачи защиты персональных данных клиентов было предложено преобразовывать данные путем умножения признаков на обратимую матрицу. Используя формулы линейной регрессии, было доказано, что такой способ преобразования не влияет на качество модели обучения, если размер обратимой матрицы совпадает с числом признаков исходных данных.

Практическое применение предложенного алгоритма показало, что качество модели обучения на исходных и преобразованных данных не меняется. Таким образом алгоритм преобразования данных путем умножения исходных признаков на обратимую матрицу может быть применен для защиты персональных данных клиентов компании.